In [22]:
from pydataset import data

import pandas as pd
import numpy as np

from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

In [4]:
df = data('tips')

In [5]:
df.head()


,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


Create a column named price_per_person. This should be the total bill divided by the party size.

In [6]:
df['price_per_person'] = df['total_bill']/df.size

Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?  

definitely total_bill first, maybe time second?

Use select k best to select the top 2 features for predicting tip amount. What are they?

In [9]:
X_train = df.drop(columns='tip')
y_train = df.tip

In [13]:
X_train.head()

,total_bill,sex,smoker,day,time,size,price_per_person
1,16.99,Female,No,Sun,Dinner,2,0.009947
2,10.34,Male,No,Sun,Dinner,3,0.006054
3,21.01,Male,No,Sun,Dinner,3,0.012301
4,23.68,Male,No,Sun,Dinner,2,0.013864
5,24.59,Female,No,Sun,Dinner,4,0.014397


In [17]:
X_train = pd.get_dummies(df, columns = ['sex','smoker','day','time','size'],drop_first=True)

In [21]:
f_selector = SelectKBest(f_regression,k=2)
f_selector.fit(X_train,y_train)
selectors = f_selector.get_support()
X_train.columns[selectors]


Index(['total_bill', 'price_per_person'], dtype='object')

Use recursive feature elimination to select the top 2 features for tip amount. What are they?

In [23]:
lm = LinearRegression()
rfe = RFE(lm,n_features_to_select=2)
rfe.fit(X_train,y_train)
mask = rfe.support_
X_train.columns[mask]

Index(['tip', 'sex_Male'], dtype='object')

Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features you are selecting?  

They give different answers because they select features using a different method. I'm sure this is variable depending on the number of features selected

### Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [28]:
def select_kbest(x,y,k):
    from sklearn.feature_selection import SelectKBest, f_regression
    f_selector = SelectKBest(f_regression,k=k)
    f_selector.fit(x,y)
    mask = f_selector.get_support()
    return x.columns[mask]

In [30]:
select_kbest(X_train,y_train,2)

Index(['total_bill', 'price_per_person'], dtype='object')

### Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [34]:
def rfe(x,y,k):
    from sklearn.linear_model import LinearRegression
    from sklearn.feature_selection import RFE
    
    lm = LinearRegression()
    rfe = RFE(lm,n_features_to_select=k)
    rfe.fit(x,y)
    
    mask = rfe.support_
    
    return x.columns[mask]
    

In [35]:
rfe(X_train,y_train,2)

Index(['tip', 'sex_Male'], dtype='object')

### Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [36]:
df_swiss = data('swiss')

In [42]:
df_swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [ ]:
rfe(df_swiss_x,df_swiss_y,3)

In [46]:
df_swiss_x = df_swiss.drop(columns = 'Fertility')
df_swiss_y = df_swiss['Fertility']

In [47]:
rfe(df_swiss_x,df_swiss_y,3)

Index(['Examination', 'Education', 'Infant.Mortality'], dtype='object')

In [48]:
select_kbest(df_swiss_x,df_swiss_y,3)

Index(['Examination', 'Education', 'Catholic'], dtype='object')